In [1]:
import datetime

import smbus
import trio
import bokeh.io
import bokeh.plotting
import bokeh as bk
import numpy as np
import RPi.GPIO as GPIO
import time
import httpx

bk.io.output_notebook()
%autoawait trio

Loading BokehJS ...

In [2]:
## I2C address select
ADDRESS_0                 = 0x70
ADDRESS_1                 = 0x71
ADDRESS_2                 = 0x72
ADDRESS_3                 = 0x73

In [3]:
address = ADDRESS_0
i2cbus = smbus.SMBus(1)
calibration_factor = (23 / 120.0)

buzzerpin = 23
ledpin_red = 24
ledpin_green = 25

## Register for oxygen data
OXYGEN_DATA_REGISTER      = 0x03
## Register for users to configure key value manually
USER_SET_REGISTER         = 0x08
## Register for automatically configuring key value
AUTO_SET_REGISTER       = 0x09
## Register for obtaining key value
GET_KEY_REGISTER          = 0x0A

In [4]:
def setup():
    GPIO.setmode(GPIO.BCM) # use LOGICAL GPIO Numbering
    GPIO.setup(ledpin_red, GPIO.OUT) # set the red ledPin to OUTPUT mode
    GPIO.output(ledpin_red, GPIO.LOW) # make red ledPin output LOW level
    
    GPIO.setup(ledpin_green, GPIO.OUT) # set the green ledPin to OUTPUT mode
    GPIO.output(ledpin_green, GPIO.HIGH) # make green ledPin output HIGH level  
    
    GPIO.setup(buzzerpin, GPIO.OUT) # set buzzerPin to OUTPUT mode

In [5]:
def aggregate_data(alldata):
    time = np.array([time for time, data in alldata])
    data = np.array([data for time, data in alldata])

    min = data.min()
    max = data.max()
    avg = data.mean()
    start_time, _ = alldata[0]

    aggregation = dict(
        time=start_time,
        min=min,
        max=max,
        avg=avg,
    )
    
    return aggregation

In [6]:
def read_oxygen(i2cbus, address):
    result = i2cbus.read_i2c_block_data(address, OXYGEN_DATA_REGISTER, 3)
    return calibration_factor * (result[0] + result[1] / 10 + result[2] / 100) # Encoding des Sensors - siehe Dokumentation

In [16]:
async def alert():
    while True:
        if low_oxygen:
            GPIO.output(ledpin_green, GPIO.LOW)
            GPIO.output(ledpin_red, GPIO.HIGH)
            GPIO.output(buzzerpin, GPIO.HIGH)
            await trio.sleep(0.5)
            GPIO.output(ledpin_red, GPIO.LOW)
            GPIO.output(buzzerpin, GPIO.LOW)
            await trio.sleep(0.5)
        else:
            GPIO.output(ledpin_green, GPIO.HIGH)
            GPIO.output(ledpin_red, GPIO.LOW)
            GPIO.output(buzzerpin, GPIO.LOW)
            await trio.sleep(0.1)

In [17]:
def fig_setup():
    fig = bk.plotting.figure(x_axis_type="datetime")
    time = np.array([], dtype=np.datetime64)
    ds = bk.models.ColumnDataSource(data=dict(time=time,oxygenlevel=[])) # like empty df
    fig.line(x="time",y="oxygenlevel",source=ds)
    handle = bk.io.show(fig,notebook_handle=True) # um Daten nachzuschieben im Notebook

    return ds, handle

In [18]:
def plot(ds, handle, time, oxygen):
    ds.stream(dict(time=[time],oxygenlevel=[oxygen]),rollover=600) # Neue Daten werden an das Ende der Kolonnen angehängt, ab 600 werden die ältesten Daten herausgeworfen
    bk.io.push_notebook(handle=handle)

In [19]:
async def measurement(*,measurement_interval=0.1, aggregation_interval=10):
    
    global low_oxygen
    ds, handle = fig_setup()
    next_measurement = trio.current_time()+measurement_interval
    next_aggregation = trio.current_time()+aggregation_interval
    
    while True:
        alldata = [] # um alle Daten zu speichern (!Überlegen wegen Memory)   
        
        while trio.current_time()<next_aggregation:      
            try:
                time = np.datetime64(datetime.datetime.now())
                oxygen = read_oxygen(i2cbus, address)
            except Exception as ex:
                print(f'{ex!r} - retry')
                await trio.sleep(0.05)
                continue
    
            alldata.append((time, oxygen))
            
            plot(ds, handle, time, oxygen)
            low_oxygen = oxygen<20
            
            await trio.sleep_until(next_measurement)
            next_measurement += measurement_interval

        next_aggregation += aggregation_interval
        aggregation = aggregate_data(alldata)
        print(aggregation)

In [20]:
low_oxygen = False

In [21]:
setup()

try: 
    async with trio.open_nursery() as nursery:
        nursery.start_soon(alert)
        nursery.start_soon(measurement)
finally:
    GPIO.cleanup()

{'time': numpy.datetime64('2024-03-06T17:19:52.837759'), 'min': 19.358333333333334, 'max': 43.95300000000001, 'avg': 20.869625000000006}


KeyboardInterrupt: 